In [15]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import time
from bs4 import BeautifulSoup
import datetime
import csv

# Set up Selenium WebDriver
chrome_options = Options()
chrome_options.add_argument("--headless")  # Run in headless mode
chrome_options.add_argument("--disable-gpu")  # Disable GPU acceleration
chrome_options.add_argument("--no-sandbox")  # Bypass OS security model
chrome_options.add_argument("--disable-dev-shm-usage")  # Overcome limited resource problems

service = Service(executable_path="C:/Users/kavya/chromedriver/chromedriver.exe")  # Update with the path to your chromedriver
driver = webdriver.Chrome(service=service, options=chrome_options)

# Open the webpage
driver.get("https://news.metal.com/list/industry/aluminium")

# Function to click "More" button and load more articles
def load_more_articles(driver, times=5):
    for _ in range(times):
        try:
            more_button = driver.find_element(By.CLASS_NAME, 'footer___PvIjk')  # Update with actual class name
            more_button.click()
            time.sleep(2)
        except:
            break

# Load more articles
load_more_articles(driver, times=10)

# Parse the page source with BeautifulSoup
soup = BeautifulSoup(driver.page_source, 'html.parser')

# Close the WebDriver
driver.quit()
# Extract articles
articles = []
for item in soup.find_all('div', class_='newsItem___wZtKx'):  # Update with actual class name
    title = item.find('div', class_='title___1baLV').text.strip()  # Update with actual class name
    description = item.find('div', class_='description___z7ktb').text.strip()  # Update with actual class name
    date_text = item.find('div', class_='date___3dzkE').text.strip()  # Update with actual class name

    # Determine the appropriate date format
    if len(date_text.split()) == 2:
        # Format: 'Month Day, Year'
        date_format = '%b %d, %Y'
    else:
        # Format: 'Month Day, Year HH:MM'
        date_format = '%b %d, %Y %H:%M'

    try:
        date = datetime.datetime.strptime(date_text, date_format)
    except ValueError:
        continue  # Skip if the date format does not match

    # Filter articles from the last 45 days
    if (datetime.datetime.now() - date).days <= 45:
        articles.append({
            'title': title,
            'description': description,
            'date': date_text
        })

# Save articles to CSV
csv_file_path = 'articles.csv'
with open(csv_file_path, 'w', newline='', encoding='utf-8') as csvfile:
    fieldnames = ['title', 'description', 'date']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

    writer.writeheader()
    for article in articles:
        writer.writerow(article)

print(f"Articles have been saved to {csv_file_path}")

Articles have been saved to articles.csv
